Get everything set up as it was for the retrain step

In [1]:
import torch

from torchvision import  models


data_dir = 'images/clean_image_data'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = torch.load('model_evaluation/feature/best_weights.pt')


Last layer is avgpool as it would be with resnet 18

In [2]:

layer = model._modules.get('avgpool')

Get the test image into the right state

In [3]:
import image_processor
pyImage = image_processor.fullPreProcess_Image(filePath='images/clean_image_data/0a1baaa8-4556-4e07-a486-599c05cce76c.jpg').unsqueeze(0)

#Create a vector of zeros that will hold our feature vector

my_embedding = torch.zeros(2048)

#Define a function that will copy the output of a layer
def copy_data(m, i, o):
    my_embedding.copy_(o.data.reshape(o.data.size(1)))
    
# Attach that function to our selected layer

h = layer.register_forward_hook(copy_data)

Run model on our image

In [4]:
model(pyImage.to(device))

# Detach our copy function from the layer
h.remove()

print(my_embedding)

tensor([1.3251, 0.8771, 0.5917,  ..., 0.0200, 0.1635, 0.8196])


This now works for a single image, so let's shunt it off into a function, then test that function

In [5]:
second_embedding = image_processor.getEmbedding(image='images/clean_image_data/0a1baaa8-4556-4e07-a486-599c05cce76c.jpg',model=model)

if torch.equal(my_embedding, second_embedding):
    print("success")

success


That now works, let's get our list of images and classify them all

In [7]:
import pandas as pd
import os
from tqdm import tqdm

imageList =   pd.read_csv(filepath_or_buffer='training_data.csv')

imageList['embedding'] =[image_processor.getEmbedding(image=os.path.join(data_dir,x +'.jpg'),model=model) for x in tqdm( imageList['id'],
                                                bar_format='{l_bar}{bar}| {percentage:3.0f}% {n}/{total} [{remaining}{postfix}]',desc="Calculating Embeddings")]

imageList.head()

Resizing: 100%|██████████| 100% 12604/12604 [00:00]


,id,label,embedding
0,912bb259-3ad9-457b-9db1-ce1da9016057,6,"[tensor(0.4134), tensor(0.3961), tensor(0.2943..."
1,b166d305-b852-4bdd-83f4-465b20da94fa,6,"[tensor(0.2127), tensor(0.1100), tensor(0.7573..."
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,6,"[tensor(0.2206), tensor(0.4567), tensor(0.5249..."
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,6,"[tensor(0.5860), tensor(1.0795), tensor(0.4006..."
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,6,"[tensor(0.2624), tensor(0.4993), tensor(0.0598..."


Get rid of those nasty tensor representations so we can serialise the data

In [14]:
imageList['embedding'] = [x.tolist() for x in imageList['embedding']]
imageList.head()

,id,label,embedding
0,912bb259-3ad9-457b-9db1-ce1da9016057,6,"[0.41342562437057495, 0.396090567111969, 0.294..."
1,b166d305-b852-4bdd-83f4-465b20da94fa,6,"[0.2126685529947281, 0.11002184450626373, 0.75..."
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,6,"[0.22057314217090607, 0.4567006826400757, 0.52..."
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,6,"[0.5859533548355103, 1.079506516456604, 0.4005..."
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,6,"[0.2624373137950897, 0.49925994873046875, 0.05..."


Throw it all into json

In [18]:
imageList.to_json("embeddings.json")

Bring it back in and check that the serialisation worked

In [22]:
newImageList = pd.read_json("embeddings.json")

print(newImageList.head().compare(imageList.head()))

print(len(newImageList))

                                           embedding  \
                                                self   
0  [0.4134256244, 0.3960905671, 0.294250667100000...   
1  [0.212668553, 0.11002184450000001, 0.757327497...   
2  [0.22057314220000002, 0.4567006826, 0.52488499...   
3  [0.5859533548, 1.0795065165, 0.4005952179, 0.1...   
4  [0.2624373138, 0.4992599487, 0.0597878285, 0.1...   

                                                      
                                               other  
0  [0.41342562437057495, 0.396090567111969, 0.294...  
1  [0.2126685529947281, 0.11002184450626373, 0.75...  
2  [0.22057314217090607, 0.4567006826400757, 0.52...  
3  [0.5859533548355103, 1.079506516456604, 0.4005...  
4  [0.2624373137950897, 0.49925994873046875, 0.05...  
12604


Long story short, the serialised version has lower floating point precision than the unserialised one.

Not the end of the world since we're finding "broad" similarity between images, but not ideal for some situations.